In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark


tar: spark-2.4.1-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col,lit,round
from pyspark.sql.types import  StructField, StructType, StringType, IntegerType,FloatType
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler,MinMaxScaler
import numpy as np
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier

In [ ]:
spark = SparkSession.builder.appName("Projeto Classificador").getOrCreate()
dataset = spark.read.csv("cardio_train.csv",sep = ";",header=True)

In [ ]:
dataset.show()

+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
| id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|  0|18393|     2|   168|  62.0|  110|   80|          1|   1|    0|   0|     1|     0|
|  1|20228|     1|   156|  85.0|  140|   90|          3|   1|    0|   0|     1|     1|
|  2|18857|     1|   165|  64.0|  130|   70|          3|   1|    0|   0|     0|     1|
|  3|17623|     2|   169|  82.0|  150|  100|          1|   1|    0|   0|     1|     1|
|  4|17474|     1|   156|  56.0|  100|   60|          1|   1|    0|   0|     0|     0|
|  8|21914|     1|   151|  67.0|  120|   80|          2|   2|    0|   0|     0|     0|
|  9|22113|     1|   157|  93.0|  130|   80|          3|   1|    0|   0|     1|     0|
| 12|22584|     2|   178|  95.0|  130|   90|          3|   3|    0|   0|     1|     1|
| 13|17668|     1|   158|  71.0|  110|   70

In [ ]:
dataset.count()

70000

In [ ]:
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- ap_hi: string (nullable = true)
 |-- ap_lo: string (nullable = true)
 |-- cholesterol: string (nullable = true)
 |-- gluc: string (nullable = true)
 |-- smoke: string (nullable = true)
 |-- alco: string (nullable = true)
 |-- active: string (nullable = true)
 |-- cardio: string (nullable = true)



In [ ]:
dataset = dataset.select([col(c).cast('Integer') for c in dataset.columns])

In [ ]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- ap_hi: integer (nullable = true)
 |-- ap_lo: integer (nullable = true)
 |-- cholesterol: integer (nullable = true)
 |-- gluc: integer (nullable = true)
 |-- smoke: integer (nullable = true)
 |-- alco: integer (nullable = true)
 |-- active: integer (nullable = true)
 |-- cardio: integer (nullable = true)



In [ ]:
dataset.show()

+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
| id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|  0|18393|     2|   168|    62|  110|   80|          1|   1|    0|   0|     1|     0|
|  1|20228|     1|   156|    85|  140|   90|          3|   1|    0|   0|     1|     1|
|  2|18857|     1|   165|    64|  130|   70|          3|   1|    0|   0|     0|     1|
|  3|17623|     2|   169|    82|  150|  100|          1|   1|    0|   0|     1|     1|
|  4|17474|     1|   156|    56|  100|   60|          1|   1|    0|   0|     0|     0|
|  8|21914|     1|   151|    67|  120|   80|          2|   2|    0|   0|     0|     0|
|  9|22113|     1|   157|    93|  130|   80|          3|   1|    0|   0|     1|     0|
| 12|22584|     2|   178|    95|  130|   90|          3|   3|    0|   0|     1|     1|
| 13|17668|     1|   158|    71|  110|   70

In [ ]:
dataset.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in dataset.columns]).show()

+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
| id|age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|  0|  0|     0|     0|     0|    0|    0|          0|   0|    0|   0|     0|     0|
+---+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+



In [ ]:
dataset.groupBy('cardio').count().show()

+------+-----+
|cardio|count|
+------+-----+
|     1|34979|
|     0|35021|
+------+-----+



In [ ]:
dataset = dataset.drop("id")

In [ ]:
dataset = dataset.withColumn("age",round((dataset.age/360)).cast("integer").alias("age"))

In [ ]:
dataset.show()

+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+---+------+------+------+-----+-----+-----------+----+-----+----+------+------+
| 51|     2|   168|    62|  110|   80|          1|   1|    0|   0|     1|     0|
| 56|     1|   156|    85|  140|   90|          3|   1|    0|   0|     1|     1|
| 52|     1|   165|    64|  130|   70|          3|   1|    0|   0|     0|     1|
| 49|     2|   169|    82|  150|  100|          1|   1|    0|   0|     1|     1|
| 49|     1|   156|    56|  100|   60|          1|   1|    0|   0|     0|     0|
| 61|     1|   151|    67|  120|   80|          2|   2|    0|   0|     0|     0|
| 61|     1|   157|    93|  130|   80|          3|   1|    0|   0|     1|     0|
| 63|     2|   178|    95|  130|   90|          3|   3|    0|   0|     1|     1|
| 49|     1|   158|    71|  110|   70|          1|   1|    0|   0|     1|     0|
| 55|     1|   164|    68|  

In [ ]:
vetor_col = "corr_features"
ass = VectorAssembler(inputCols=dataset.columns, outputCol=vetor_col)
df_vetor = ass.transform(dataset).select(vetor_col)
matrix = Correlation.corr(df_vetor, vetor_col)

In [ ]:
matrix = matrix.collect()[0][0].values
matrix_cor = np.reshape(matrix,(12,12))

In [ ]:
dataset.groupby("age").count().sort("age").show()

+---+-----+
|age|count|
+---+-----+
| 30|    4|
| 40| 1580|
| 41| 1821|
| 42| 1635|
| 43| 1682|
| 44| 1642|
| 45| 1905|
| 46| 1608|
| 47| 2101|
| 48| 1538|
| 49| 2458|
| 50| 2210|
| 51| 4377|
| 52| 2125|
| 53| 4506|
| 54| 2209|
| 55| 5222|
| 56| 2143|
| 57| 5327|
| 58| 1906|
+---+-----+
only showing top 20 rows



In [ ]:

schema = StructType([
    StructField('age', FloatType(), True),
    StructField('gender',  FloatType(), True),
    StructField('height',  FloatType(), True),
    StructField('weight',  FloatType(), True),
    StructField('ap_hi',  FloatType(), True),
    StructField('ap_lo',  FloatType(), True),
    StructField('cholesterol',  FloatType(), True),
    StructField('gluc',  FloatType(), True),
    StructField('smoke',  FloatType(), True),
    StructField('alco',  FloatType(), True),
    StructField('active',  FloatType(), True),
    StructField('cardio',  FloatType(), True)
])

rdd = spark.sparkContext.parallelize(matrix_cor.tolist())
df_cor = spark.createDataFrame(rdd,schema)
df_cor.show()

+------------+------------+------------+-----------+-------------+------------+------------+-------------+-------------+------------+-------------+------------+
|         age|      gender|      height|     weight|        ap_hi|       ap_lo| cholesterol|         gluc|        smoke|        alco|       active|      cardio|
+------------+------------+------------+-----------+-------------+------------+------------+-------------+-------------+------------+-------------+------------+
|         1.0|-0.022684608|-0.081389904|0.053550016|  0.020465912| 0.017431136|  0.15433328|   0.09837528| -0.047555476|-0.029837351| -0.010087403|  0.23765938|
|-0.022684608|         1.0|  0.49903342| 0.15541604| 0.0060053086| 0.015253971| -0.03582141| -0.020491036|   0.33813512|  0.17096643| 0.0058656693| 0.008109492|
|-0.081389904|  0.49903342|         1.0| 0.29100123| 0.0054877643|0.0061504673| -0.05022595| -0.018594671|    0.1879894|  0.09441869| -0.006569547|-0.010821058|
| 0.053550016|  0.15541604|  0.291

In [ ]:
dataset = dataset.withColumn("label",dataset["cardio"])
dataset = dataset.drop("cardio")
x = dataset.drop("label")
x = x.drop("alco",'gender')

In [ ]:
df = VectorAssembler(inputCols=x.columns, outputCol="features")
df = df.transform(dataset)

In [ ]:
df.show()

+---+------+------+------+-----+-----+-----------+----+-----+----+------+-----+--------------------+
|age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|label|            features|
+---+------+------+------+-----+-----+-----------+----+-----+----+------+-----+--------------------+
| 51|     2|   168|    62|  110|   80|          1|   1|    0|   0|     1|    0|[51.0,168.0,62.0,...|
| 56|     1|   156|    85|  140|   90|          3|   1|    0|   0|     1|    1|[56.0,156.0,85.0,...|
| 52|     1|   165|    64|  130|   70|          3|   1|    0|   0|     0|    1|[52.0,165.0,64.0,...|
| 49|     2|   169|    82|  150|  100|          1|   1|    0|   0|     1|    1|[49.0,169.0,82.0,...|
| 49|     1|   156|    56|  100|   60|          1|   1|    0|   0|     0|    0|[49.0,156.0,56.0,...|
| 61|     1|   151|    67|  120|   80|          2|   2|    0|   0|     0|    0|[61.0,151.0,67.0,...|
| 61|     1|   157|    93|  130|   80|          3|   1|    0|   0|     1|    0|[61.0,157.0,

In [ ]:
featureScaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures").fit(df)

In [ ]:
(treino, teste) = df.randomSplit([0.7, 0.3])

In [ ]:
gbt = GBTClassifier(labelCol="label",maxIter=10)

In [ ]:
pipeline = Pipeline(stages=[ gbt])

In [ ]:
modelo = pipeline.fit(treino)

pred = modelo.transform(teste)

In [ ]:
pred.show()

+---+------+------+------+-----+-----+-----------+----+-----+----+------+-----+--------------------+--------------------+--------------------+----------+
|age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|label|            features|       rawPrediction|         probability|prediction|
+---+------+------+------+-----+-----+-----------+----+-----+----+------+-----+--------------------+--------------------+--------------------+----------+
| 40|     1|   120|    80|  120|   80|          1|   1|    0|   0|     0|    1|[40.0,120.0,80.0,...|[0.90632470176242...|[0.85968177387943...|       0.0|
| 40|     1|   142|    57|  110|   80|          1|   3|    0|   0|     0|    0|[40.0,142.0,57.0,...|[0.86379758062109...|[0.84910455647505...|       0.0|
| 40|     1|   150|    55|  110|   70|          1|   1|    0|   0|     1|    0|[40.0,150.0,55.0,...|[1.01327058071754...|[0.88355568704060...|       0.0|
| 40|     1|   150|    60|  110|   70|          1|   1|    0|   0|     1|   

In [ ]:
pred.select("rawPrediction","prediction", "label", "features").show(15)


evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="label")
accuracy = evaluator.evaluate(pred,{evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(pred,{evaluator.metricName: "f1"})
print(accuracy)
print(f1)




+--------------------+----------+-----+--------------------+
|       rawPrediction|prediction|label|            features|
+--------------------+----------+-----+--------------------+
|[0.90632470176242...|       0.0|    1|[40.0,120.0,80.0,...|
|[0.86379758062109...|       0.0|    0|[40.0,142.0,57.0,...|
|[1.01327058071754...|       0.0|    0|[40.0,150.0,55.0,...|
|[0.95606553927082...|       0.0|    0|[40.0,150.0,60.0,...|
|[1.00664128526827...|       0.0|    0|[40.0,150.0,61.0,...|
|[0.89322345960210...|       0.0|    0|[40.0,150.0,65.0,...|
|[-0.7414266624841...|       1.0|    0|[40.0,150.0,85.0,...|
|[0.96188138398986...|       0.0|    0|[40.0,151.0,40.0,...|
|[1.01327058071754...|       0.0|    0|[40.0,151.0,50.0,...|
|[0.86741857463597...|       0.0|    1|[40.0,151.0,71.0,...|
|[0.52164886995997...|       0.0|    0|[40.0,152.0,54.0,...|
|[-0.5734488868336...|       1.0|    1|[40.0,152.0,54.0,...|
|[1.00664128526827...|       0.0|    0|[40.0,152.0,67.0,...|
|[0.94448977516700...|  

In [ ]:
df.groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|34979|
|    0|35021|
+-----+-----+



In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6,8,10])
             .addGrid(gbt.maxBins, [20,30,40, 50,60])
             .addGrid(gbt.maxIter, [5,10, 20,30])
             .build())
gbtEval = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=gbtEval, numFolds=5)

cvModel = cv.fit(treino)
cvPreds = cvModel.transform(teste)
gbtEval.evaluate(cvPreds)

0.8025732025988123